<a href="https://colab.research.google.com/github/Eunhye1109/Practical-Project/blob/EH/API_%3EOpen_AI_Sum_%3E%EC%A0%80%EC%9E%A5_250728.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install requests PyMuPDF openai python-dotenv tqdm


In [28]:
import os
import io
import re
import zipfile
import requests
import fitz  # PyMuPDF
from dotenv import load_dotenv
from tqdm import tqdm
import openai
import pandas as pd
import requests
import zipfile
from tqdm import tqdm


# ---------------------------
# 0. 환경 변수 로드 & 설정
# ---------------------------
load_dotenv()
DART_API_KEY = os.getenv("DART_API_KEY", "입력하세요")  # 사용자가 제공한 키
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "입력하세요")  # 본인 키 입력 or .env
openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)

# ---------------------------

# 0. corp_code 다운로드 및 변환
import requests, zipfile, xml.etree.ElementTree as ET
import pandas as pd

def download_corp_code(api_key: str, save_path: str = "CORPCODE.csv"):
    url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}"
    zip_path = "corp_code.zip"

# ---------------------------


In [38]:
# 1. DART 원본 문서(zip) 다운로드
# ---------------------------
def download_dart_document_zip(crtfc_key: str, rcept_no: str) -> bytes:
    """
    DART document.xml 엔드포인트로부터 zip 바이너리를 받아서 bytes로 반환.
    에러가 나면 status/message를 출력하고 예외 발생.
    """
    url = "https://opendart.fss.or.kr/api/document.xml"
    params = {
        "crtfc_key": crtfc_key,
        "rcept_no": rcept_no
    }
    resp = requests.get(url, params=params, stream=True)
    if resp.status_code != 200:
        raise RuntimeError(f"HTTP {resp.status_code} - {resp.text[:500]}")
    content = resp.content
    if not content.startswith(b'PK'):
        text = content.decode('utf-8', errors='ignore')
        status = re.search(r"<status>(.*?)</status>", text)
        message = re.search(r"<message>(.*?)</message>", text)
        s = status.group(1) if status else "UNKNOWN"
        m = message.group(1) if message else text[:200]
        raise RuntimeError(f"DART API error - status: {s}, message: {m}")
    return content

# ---------------------------
# 2. Zip 해제 & PDF/XML 파일 추출
# ---------------------------
def extract_documents_from_zip(zip_bytes: bytes, out_dir: str = None):
    pdf_files = []
    xml_files = []
    with zipfile.ZipFile(io.BytesIO(zip_bytes)) as zf:
        for name in zf.namelist():
            data = zf.read(name)
            if name.lower().endswith(".pdf"):
                pdf_files.append((name, data))
            elif name.lower().endswith(".xml"):
                xml_files.append((name, data))
            if out_dir:
                os.makedirs(out_dir, exist_ok=True)
                with open(os.path.join(out_dir, os.path.basename(name)), "wb") as f:
                    f.write(data)
    return pdf_files, xml_files

# ---------------------------
# 3. PDF 및 XML 텍스트 추출
# ---------------------------
def pdf_bytes_to_text(pdf_bytes: bytes) -> str:
    with fitz.open(stream=pdf_bytes, filetype="pdf") as doc:
        return "".join([page.get_text() for page in doc])

def xml_bytes_to_text(xml_bytes: bytes) -> str:
    text = xml_bytes.decode("utf-8", errors="ignore")
    return re.sub(r"<[^>]+>", " ", text)

# ---------------------------
# 4. 전처리 및 요약 로직 동일
# ---------------------------
# (이전 전처리/토픽/요약 관련 코드 동일 유지)

# ---------------------------
# 5. 메인 플로우
# ---------------------------
if __name__ == "__main__":
    company_name = input("기업명을 입력하세요: ").strip()
    rcept_no = get_receipt_number_by_company(company_name, DART_API_KEY)  # 필수 인자 추가

    print("📥 Zip 다운로드 중...")
    zip_bytes = download_dart_document_zip(DART_API_KEY, rcept_no)

    print("🗂 PDF/XML 추출 중...")
    pdf_files, xml_files = extract_documents_from_zip(zip_bytes)
    print("📦 압축 파일 내부 목록:")
    for name, _ in pdf_files + xml_files:
        print(f" - {name}")

    all_text = []
    if pdf_files:
        print("📄 PDF 텍스트 추출 중...")
        for name, data in pdf_files:
            try:
                t = pdf_bytes_to_text(data)
                if t.strip():
                    all_text.append(t)
            except Exception as e:
                print(f"⚠️ {name} 추출 실패: {e}")
    elif xml_files:
        print("📄 XML 텍스트 추출 중...")
        for name, data in xml_files:
            try:
                t = xml_bytes_to_text(data)
                if t.strip():
                    all_text.append(t)
            except Exception as e:
                print(f"⚠️ {name} 추출 실패: {e}")
    else:
        raise RuntimeError("❌ ZIP 내에 PDF 또는 XML 파일이 없습니다.")

    merged_text = preprocess_text("\n".join(all_text))
    sentences = [s for s in split_sentences(merged_text) if is_valid_sentence(s)]
    print(f"📚 전처리 후 문장 수: {len(sentences)}")

    results = summarize_all(company_name, sentences)

   # ✅ 파일 저장 경로에 기업명 및 API 호출 정보 포함
save_path = f"{rcept_no}_{company_name}_GPT요약_API_호출.txt"
with open(save_path, "w", encoding="utf-8") as f:
    for sec, summ in results.items():
        if "해당 토픽 관련 정보 없음." in summ:
            continue
        f.write(f"\n\n## {sec}\n{summ}")

print(f"\n✅ 요약 저장 경로: {os.path.abspath(save_path)}")

# ✅ 저장 파일 내용 출력
with open(save_path, "r", encoding="utf-8") as f:
    print(f.read())

# ✅ 압축 해제된 파일 경로 출력
print("\n📁 압축 해제된 파일 경로:")
print(f"→ 현재 디렉토리 기준: {os.getcwd()}")
if pdf_files or xml_files:
    for name, _ in pdf_files + xml_files:
        print(f"   - {os.path.abspath(os.path.join(os.getcwd(), name))}")


기업명을 입력하세요: LG
📥 Zip 다운로드 중...
🗂 PDF/XML 추출 중...
📦 압축 파일 내부 목록:
 - 20250728000359.xml
📄 XML 텍스트 추출 중...
📚 전처리 후 문장 수: 849


Summarizing by topic: 100%|██████████| 5/5 [00:23<00:00,  4.76s/it]


✅ 요약 저장 경로: /content/20250728000359_LG_GPT요약_API_호출.txt


## 📊 재무 요약
최근 3개년 동안 LG의 매출액은 꾸준히 증가하여 2022년 262,522백만원, 2023년 278,943백만원, 2024년 288,707백만원을 기록했습니다. 영업이익은 2022년 48,213백만원에서 2023년 54,950백만원으로 상승한 후, 2024년에는 56,283백만원으로 증가했습니다. 당기순이익은 2022년 45,275백만원에서 2023년 48,262백만원으로 증가했으나, 2024년에는 32,453백만원으로 감소했습니다. 이러한 재무 성과는 LG의 지속적인 연구개발 투자와 개량신약 개발에 대한 집중적인 노력이 반영된 결과로 보입니다.

## 🛡️ 리스크 및 부채
LG는 변동이자부 장기차입금으로 인해 시장이자율 변동위험에 노출되어 있으며, 이를 관리하기 위해 고정이자율차입금과 변동이자율차입금의 균형을 유지하고 있습니다. 유동부채는 최근 48,195백만원으로 감소하였으며, 비유동부채는 19,527백만원으로 증가했습니다. 부채총계는 67,722백만원으로 이전보다 감소하였습니다. 금융자산과 부채는 공정가치로 평가되며, 유동성 리스크 관리를 위해 자산과 부채를 유동/비유동으로 구분하여 관리하고 있습니다. 담보 제공 및 지급보증과 관련된 구체적인 수치는 제공되지 않았습니다.

## 🚀 사업 전략
LG의 사업보고서에 따르면, 회사는 지속적인 R&D 투자를 통해 개량신약 및 신 제제 개선품목을 출시하고 있으며, 매출액 대비 11% 이상의 연구개발비를 투자하고 있습니다. 또한, 첨단 생산 설비 및 시설에 대한 투자를 지속하여 최신 설비와 장비를 구축하고, 우수한 품질의 의약품 생산에 집중하고 있습니다. LG는 신약 개발에도 지속적으로 투자하고 있으며, 이러한 노력을 통해 업계 최상위 수준의 R&D 능력을 유지하고 있습니다.

## 🔬 기술 및 R&D
LG는 연구개발(R&D)에 매출액 대비 11% 이상을 지속적으로 투자하며, 총 17개의 개량

In [34]:

    # ✅ 압축 해제된 파일 경로 출력
    print("\n📁 압축 해제된 파일 경로:")
    print(f"→ 현재 디렉토리 기준: {os.getcwd()}")
    if pdf_files or xml_files:
        for name, _ in pdf_files + xml_files:
            print(f"   - {os.path.abspath(os.path.join(os.getcwd(), name))}")

    # ✅ 요약 결과 저장 경로 출력
    print(f"\n📝 요약 결과 저장 경로: {os.path.abspath(save_path)}")



📁 압축 해제된 파일 경로:
→ 현재 디렉토리 기준: /content
   - /content/20250728000359.xml

📝 요약 결과 저장 경로: /content/20250728000359_GPT요약.txt
